# Loss는 어디서 생길까?

먼저 다음을 가정하겠습니다.

* input(입력) = x
* Output(출력) = y
* Label(정답) = d

1. 우리는 입력과 정답만 가지고 있습니다.  

2. 모델은 입력을 받아서 출력을 만듭니다. -> y = wx + b 

3. 우리는 d - y가 0이길 희망합니다. 

4. d - y != 0이면 문제가 있다고 봅니다 -> Error --> **Loss**

***

# Error(=Loss)를 어떻게 계산하지? 

### 1. 회귀(Regression) 

#### 1) 출력층에 쓰이는 활성화 함수
* 항등사항(=나오는대로 씁니다.) 

tensorflow로는
```python
H = tf.matmul(X,W) + b
```
라고 보통 씁니다.

#### 2) Loss Function
* MSE(Mean Squared Error)

tensorflow로는 
```python
cost = tf.reduce_mean(tf.square(H - Y))
```
라고 보통 씁니다.

### 2. 이중분류(Binary Classification)

#### 1) 출력층에 쓰이는 활성화 함수
- Logistic Function
- Sigmoid
- Tanh
- Relu
- Leaky Relu

tensorflow로는
```python
H = tf.sigmoid(tf.matmul(X, W) + b)
```
라고 보통 씁니다.

#### 2) Loss Function
- Binary Cross Entropy Loss

tensorflow로는
```python
cost = -tf.reduce_mean(Y*tf.log(H) + (1 - Y)*tf.log(1 - H))
```
라고 보통 씁니다.

### 3. 다중분류(Multinomial Classification) 

#### 1) 출력층에 쓰이는 활성화 함수
- Softmax

tensorflow로는
```python
H = tf.nn.softmax(logits)
```
라고 보통 씁니다.

#### 2) Loss Function
- (Multinomial) Cross Entropy 

tensorflow로는
```python
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(H), axis=1))
```
또는 
```python
# 아래 코드로 할 때는 'H = tf.nn.softmax(logits)' 안해도 됩니다.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=H, labels=Y))
```
라고 보통 씁니다.

***

# Loss Functions (torch.nn.xxxx)

*은 많이 쓰이는 것을 표현하였습니다.

- L1Loss *
- MSELoss *
- CrossEntropyLoss *
- NLL(negative log likelihood)Loss
- PoissonNLLLoss
- NLLLoss2d
- KLDivLoss
- BCELoss *
- BCEWithLogitsLoss
- MarginRankingLoss
- HingeEmbeddingLoss
- MultiLabelMarginLoss
- SmoothL1Loss
- SoftMarginLoss
- MultiLabelSoftMarginLoss
- CosineEmbeddingLoss
- MultiMarginLoss
- TripletMarginLoss

***

# Optimizer (torch.optim.xxxx)

*은 많이 쓰이는 것을 표현하였습니다.

- Adadelta
- Adagrad *
- Adam *
- SparseAdam
- Adamax
- ASGD
- LBFGS 
- RMSprop *
- Rprop
- SGD *

***

# Loss, Optim을 사용해보자

먼저 코드진행의 순서를 살펴보겠습니다.

1. Import 하기 (import torch, torch.nn, torchvision, torch 등등)
2. Dataset 만들기 (torchvision, torch.utils.data.Dataset & Loader)
3. Model 만들기 (class MyNetwork(nn.Module): ~~)
4. Optim과 Loss 계산 함수 결정하기
5. 학습을 위한 반복문 작성하기
6. 평가 및 모델 저장

저는 여기서 CIFAR10으로 할 것이기 때문에 
- Loss Function -> CrossEntropyLoss
- Optimizer -> SGD 

으로 진행하겠습니다.

```python
loss = nn.CrossEntropyLoss()
optim = torch.optim.SGD(Net.parameters(), lr=0.001, momentum=0.9)
```

```python
for epoch in range(num_epoch):
    for i, data in enumerate(train_loader, 0):
        
        # 학습이 되기전에 0으로 초기화 해주는 부분입니다.
        optim.zero_grad()
        
        input, labels = data, testloader
        input, labels = Variable(input), Variable(labels)

        out = Net(input)

        loss = loss(out, labels)
        
        # 자동으로 각 parameters(w, b)들이 Gradient값을 가집니다.
        loss.backward()
        
        # 그 값을 가지고 parameters(w, b)들을 업데이트를 진행합니다. 
        optim.step()
```

***

# 직접 짜보자

### 1. Import

In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader 
import numpy as np 

import torchvision 
import torchvision.transforms as transforms 

import matplotlib 
import matplotlib.pyplot as plt 

### 2. Dataset 만들기

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10_data',
                                                            train=True,
                                                            download=True,
                                                            transform=transform)
 
testset = torchvision.datasets.CIFAR10(root='./cifar10_data',
                                                            train=False,
                                                            download=True,
                                                            transform=transform)

trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### 3. Model 만들기

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv2 = nn.Conv2d(64, 30, 5)
        self.fc1 = nn.Linear(30*5*5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x), inplace=True)
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.conv2(x), inplace=True)
        x = F.max_pool2d(x, (2,2))
        
        # view에 대해서 설명을 하면 batch_size x 30 x 5 x 5이기 때문에 batch_size.(30 x 5 x 5)이런 식으로 됩니다.
        # 앞에 batch_size를 통으로 fc1에 두기 위해서 하는 것 같습니다.
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)
        return x

# GPU
# net = Net().cuda()
net = Net()

### 4. Optim과 Loss 계산 함수 결정

In [4]:
cost = nn.CrossEntropyLoss()

# torch.optim.SGD(params) 여기에 model.parameter()를 반드시 해줘야합니다.
optim = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### 5. 학습을 위한 반복문 작성하기

In [5]:
num_epoch = 3

for epoch in range(num_epoch):
    for i, data in enumerate(trainloader):
        inputs, labels = data
        
        inputs, labels = Variable(inputs), Variable(labels)
        # GPU
        # inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

        # 학습이 되기전에 0으로 초기화 해주는 부분
        optim.zero_grad()
        out = net(inputs)

        loss = cost(out, labels)
        
        # 자동으로 각 parameters(w, b)들이 Gradient값을 가진다.
        loss.backward()
        
        # 그 값을 가지고 parameters(w, b)들을 업데이트를 진행한다. 
        optim.step()

        if i % 64 == 0:
            print("%d=> loss : %.3f"%(i, loss))

print("train over")

0=> loss : 2.308
64=> loss : 2.317
128=> loss : 2.310
192=> loss : 2.243
256=> loss : 2.226
320=> loss : 2.267
384=> loss : 2.200
448=> loss : 2.190
512=> loss : 2.227
576=> loss : 2.241
640=> loss : 1.817
704=> loss : 1.692
768=> loss : 1.900
832=> loss : 2.377
896=> loss : 2.108
960=> loss : 1.967
1024=> loss : 1.961
1088=> loss : 1.776
1152=> loss : 2.003
1216=> loss : 1.721
1280=> loss : 2.548
1344=> loss : 1.941
1408=> loss : 1.937
1472=> loss : 2.297
1536=> loss : 1.767
1600=> loss : 1.979
1664=> loss : 2.186
1728=> loss : 2.052
1792=> loss : 1.528
1856=> loss : 1.609
1920=> loss : 1.752
1984=> loss : 2.582
2048=> loss : 2.346
2112=> loss : 1.965
2176=> loss : 1.446
2240=> loss : 1.588
2304=> loss : 1.295
2368=> loss : 1.149
2432=> loss : 2.486
2496=> loss : 1.959
2560=> loss : 1.195
2624=> loss : 1.980
2688=> loss : 1.935
2752=> loss : 1.578
2816=> loss : 1.868
2880=> loss : 1.405
2944=> loss : 1.367
3008=> loss : 1.876
3072=> loss : 1.597
3136=> loss : 1.584
3200=> loss : 1.480

### 6. 평가 및 모델 저장

In [6]:
total = 0
correct = 0

for data in testloader:
    images, labels = data 
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1) # torch.max( _, 1)은 각 행에서 가장 큰 수만 뽑아내는 것입니다.
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %f', (100*correct/total))

Accuracy of the network on the 10000 test images: %f 63.89
